<a href="https://colab.research.google.com/github/ozgurnsahin/llm_finetuning/blob/main/deepseek_r1_traning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers
!pip install -U datasets
!pip install -q torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [2]:
import torch
import transformers
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

GPU: Tesla T4
Memory: 15.8 GB


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType
import gc

In [4]:
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [5]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    torch_dtype = torch.float16,
    device_map="auto",
    trust_remote_code=True,)

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [7]:
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training

In [8]:
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

In [9]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 36,929,536 || all params: 1,814,017,536 || trainable%: 2.0358


In [10]:
print(f"Model loaded: {model.num_parameters() / 1e6:.1f}M parameters")

Model loaded: 1814.0M parameters


In [11]:
from datasets import load_dataset

In [12]:
dataset = load_dataset("microsoft/orca-math-word-problems-200k", split="train[:3000]")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/84.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/200035 [00:00<?, ? examples/s]

In [13]:
print(f"Dataset size: {len(dataset)}")
print("Sample:", dataset[0])

Dataset size: 3000
Sample: {'question': 'Jungkook is the 5th place. Find the number of people who crossed the finish line faster than Jungkook.', 'answer': 'If Jungkook is in 5th place, then 4 people crossed the finish line faster than him.'}


In [14]:
def format_math_qa(example):
    question = example["question"]
    answer = example["answer"]

    prompt = f"<|user|>\n{question}\n<|assistant|>\n{answer}<|end|>"

    return {"text": prompt}

In [15]:
formatted_dataset = dataset.map(format_math_qa, remove_columns=dataset.column_names)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [16]:
train_val_split = formatted_dataset.train_test_split(seed=42,test_size=0.2)

In [17]:
train_dataset =  train_val_split['train']
eval_dataset =  train_val_split['test']

In [18]:
def tokenize_function(examples):
    result = tokenizer(
        examples["text"],
        truncation=True,
        padding=True,
        max_length=512,
        return_tensors="pt",
        return_overflowing_tokens=False,
    )
    result["labels"] = result["input_ids"].clone()
    return result

In [19]:
train_dataset = train_val_split["train"].map(
    tokenize_function,
    batched=True,
    remove_columns=train_val_split["train"].column_names,
    desc="Tokenizing training data"
)

eval_dataset = train_val_split["test"].map(
    tokenize_function,
    batched=True,
    remove_columns=train_val_split["test"].column_names,
    desc="Tokenizing evaluation data"
)

Tokenizing training data:   0%|          | 0/2400 [00:00<?, ? examples/s]

Tokenizing evaluation data:   0%|          | 0/600 [00:00<?, ? examples/s]

In [20]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

In [21]:
training_args = TrainingArguments(
    output_dir="./deepseek-r1-finetuned",
    num_train_epochs=2,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    dataloader_pin_memory=False,
    dataloader_num_workers=0,
    warmup_steps=100,
    learning_rate=2e-4,
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    logging_steps=25,
    eval_strategy="steps",
    eval_steps=200,
    save_strategy="steps",
    save_steps=400,
    save_total_limit=2,
    load_best_model_at_end=True,
    remove_unused_columns=False,
    report_to=None,
    seed=42,
    bf16=True,
)

In [22]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

In [23]:
trainer = Trainer(model=model,
                  args=training_args,
                  train_dataset=train_dataset,
                  eval_dataset=eval_dataset,
                  data_collator=data_collator,
                  tokenizer=tokenizer)

/tmp/ipython-input-23-4171194248.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model,
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [24]:
import os
import gc

gc.collect()
torch.cuda.empty_cache()
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = 'expandable_segments:True'

In [25]:
try:
    train_result = trainer.train()

    print("\n Training completed!")
    print(f"Final training loss: {train_result.metrics['train_loss']:.4f}")
    print(f"Training time: {train_result.metrics['train_runtime']:.2f} seconds")
    print(f"Samples/second: {train_result.metrics['train_samples_per_second']:.2f}")

except Exception as e:
    print(f"Training failed: {e}")
    raise

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nazm-albatros (nazm-albatros-techtetech) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss
200,0.407500,0.467662



 Training completed!
Final training loss: 0.4882
Training time: 8885.08 seconds
Samples/second: 0.54


In [26]:
trainer.save_model("./deepseek-r1-math-adapter")
tokenizer.save_pretrained("./deepseek-r1-math-adapter")

('./deepseek-r1-math-adapter/tokenizer_config.json',
 './deepseek-r1-math-adapter/special_tokens_map.json',
 './deepseek-r1-math-adapter/chat_template.jinja',
 './deepseek-r1-math-adapter/tokenizer.json')

In [30]:
def generate_math_response(prompt, max_length=300):
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=max_length,
            temperature=0.1,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response[len(tokenizer.decode(inputs[0], skip_special_tokens=True)):]
    return response.strip()

In [31]:
test_problems = [
    "<|user|>\nWhat is 25 + 37?\n<|assistant|>\n",
    "<|user|>\nIf a pizza costs $12 and I want to buy 3 pizzas, how much will I pay?\n<|assistant|>\n",
    "<|user|>\nSolve for x: 2x + 5 = 15\n<|assistant|>\n"
]

In [32]:
for i, problem in enumerate(test_problems, 1):
    user_question = problem.split("<|user|>")[1].split("<|assistant|>")[0].strip()
    print(f"\nTest {i}: {user_question}")
    print("-" * 40)

    try:
        response = generate_math_response(problem)
        print(f"Model: {response}")
    except Exception as e:
        print(f"Error: {e}")


Test 1: What is 25 + 37?
----------------------------------------
Model: 25 + 37 equals 62.<|end|>

Test 2: If a pizza costs $12 and I want to buy 3 pizzas, how much will I pay?
----------------------------------------
Model: If one pizza costs $12, then three pizzas would cost:

3 pizzas * $12 per pizza = $36

So, you will pay $36 for three pizzas.<|end|>

Test 3: Solve for x: 2x + 5 = 15
----------------------------------------
Model: To solve for x, you need to isolate x on one side of the equation. Here's how you do it:

1. Subtract 5 from both sides of the equation to get rid of the constant term on the left side:
   2x + 5 - 5 = 15 - 5
   2x = 10

2. Now, divide both sides of the equation by 2 to solve for x:
   2x / 2 = 10 / 2
   x = 5

So, the solution to the equation 2x + 5 = 15 is x = 5.<|end|>
